In [1]:
from dataclasses import dataclass


@dataclass
class Tape:
    
    tape: dict[int, str]
    blank : str = " "
    current : int = None
    
    def __getitem__(self,index):
        return self.tape[index] if index in self.tape else self.blank
    def __setitem__(self, index, s):
        self.tape[index] = s
    def idx_range(self):
        return min(self.tape.keys()), max(self.tape.keys())+1
    
    def __str__(self):
        t = ""
        
        for i in range(*self.idx_range()):
            t += self.tape[i]
        if self.current is not None:
            try:
                t = f"{t[:self.current]}[{t[self.current]}]{t[self.current+1:]}"
            except:
                t = f"{t}...[]"
        return t
            
class TuringMachine:
    def __init__(self, tape = None, blank=" ", initial_state = "s", 
                 final_states = None, transition_function = None,
                 starting_pos = None, max_steps = 300):
        if tape is None:
            tape = Tape(dict())
        elif type(tape) == str:
            tape = Tape(dict(enumerate(list(tape))))
        if final_states is None:
            final_states = set()
        if transition_function is None:
            transition_function = dict()
        self.max_steps = max_steps
        self.processed_steps = 0
        self.initial_state = initial_state 
        self.current_state = initial_state
        self.current_position = starting_pos or 0
        self.tape = tape
        self.final_states = final_states
        self.transition_function = transition_function
        self.mov_dict = {"R":1,
                         "L":-1,
                         "N":0}
        
    def __next__(self):
        current = self.tape[self.current_position]
        current_conf = (self.current_state, current)
        if current_conf in self.transition_function:
            t = self.transition_function[current_conf] # [state, char, mov]
            self.tape[self.current_position] = t[1]
            self.current_position += self.mov_dict[t[2]]
            self.current_state = t[0]
        self.processed_steps+=1
            
    def process(self, string):
        self.processed_steps = 0
        self.tape = Tape(dict(enumerate(list(string))))
        while (self.current_state not in self.final_states) and self.processed_steps<self.max_steps:
            next(self)
            self.processed_steps+=1
        tape = self.tape
        self.reset()
        return str(tape)
    
    def debug(self, string, step=1, log_func = lambda *x : print(*x)):
        self.processed_steps = 0
        self.tape = Tape(dict(enumerate(list(string))))
        flag = True
        
        while flag:
            self.tape.current = self.current_position
            log_func(self.current_state, self.tape)
            for i in range(step):
                if (self.current_state in self.final_states) or self.processed_steps>self.max_steps:
                    flag = False
                    break
                self.processed_steps+=1
                next(self)
            
        tape = self.tape
        self.reset()
        return str(tape)
    
    def reset(self):
        self.processed_steps = 0
        self.current = None
        self.current_position = 0
        self.tape = None
        self.current_state = self.initial_state

def to_unary(n):
    return "1"*n
def from_unary(n):
    return n.count("1")
def from_unary_div(n, div):
    q,r = n.replace("[","").replace("]","").split("=")
    return {"div":from_unary(q)/div+from_unary(r),
            "int_div":from_unary(r),
            "module":from_unary(q)}

In [2]:
initial_state = "s"
transition_function = {("s","0"):("s", "1", "R"),
                       ("s","1"):("s", "0", "R"),
                       ("s"," "):("f"," ", "N"),
                       }
final_states = {"f"}

t = TuringMachine(
                  initial_state = "s",
                  final_states = final_states,
                  transition_function=transition_function)

In [3]:
t.process("010011001 ")

'101100110 '

In [31]:
#t.debug("010011001 ")

In [5]:

transition_function = {("0","0"):("0", "0", "R"),
                       ("0","1"):("0", "1", "R"),
                       ("0"," "):("1"," ", "R"),
                       
                       ("1","0"):("1", "0", "R"),
                       ("1","1"):("1", "1", "R"),
                       ("1"," "):("2"," ", "L"),
                       
                       ("2","0"):("2", "1", "L"),
                       ("2","1"):("3", "0", "L"),
                       ("2"," "):("5"," ", "R"),
                       
                       ("3","0"):("3", "0", "L"),
                       ("3","1"):("3", "1", "L"),
                       ("3"," "):("4"," ", "L"),
                       
                       ("4","0"):("0", "1", "R"),
                       ("4","1"):("4", "0", "L"),
                       ("4"," "):("0","1", "R"),
                       
                       ("5","1"):("5", " ", "R"),
                       ("5"," "):("f", " ", "N"),

                       }
final_states = {"f"}

t = TuringMachine(
                  initial_state = "0",
                  final_states = final_states,
                  transition_function=transition_function, 
                  max_steps = 100000)

In [30]:
#t.debug("11 11111")

In [7]:
transition_function = {
                       ("0","1"):("0", "1", "R"),
                       ("0"," "):("1"," ", "R"),
                       
                       ("1","1"):("1", "1", "R"),
                       ("1"," "):("2"," ", "L"),
                       
                       ("2","1"):("3", " ", "L"),
                       ("2"," "):("f"," ", "N"),
    
                        ("3","1"):("3","1", "L"),
                        ("3"," "):("4"," ", "L"),
    
    
                     ("4","1"):("4","1", "L"),
                        ("4"," "):("5"," ", "R"),
    
                     ("5","1"):("0"," ", "R"),
                        ("5"," "):("f"," ", "N"),


                       }
final_states = {"f"}

t = TuringMachine(
                  initial_state = "0",
                  final_states = final_states,
                  transition_function=transition_function, 
                  max_steps = 100000)

In [39]:

#from_unary(t.debug(f"{to_unary(7)} {to_unary(3)}"))

In [37]:
transition_function = {
                       ("0","1"):("1", " ", "R"),
                       ("0"," "):("12"," ", "L"),
                       
                       ("1","1"):("1", "1", "R"),
                       ("1"," "):("2"," ", "R"),
                       
                       ("2","1"):("4", "Y", "R"),
                       ("2","="):("3","=", "L"),
    
                        ("3","Y"):("3","1", "L"),
                        ("3"," "):("9"," ", "L"),
    
    
                     ("4","1"):("4","1", "R"),
                        ("4","="):("5","=", "R"),
    
                     ("5","1"):("5","1", "R"),
                        ("5"," "):("6","1", "L"),
                    
                    ("6","1"):("6","1", "L"),
                        ("6","="):("7","=", "L"),
                        
                        ("7","1"):("7","1", "L"),
                        ("7","Y"):("8","Y", "R"),
    
                       ("8","="):("3","=", "L"),
                       ("8","1"):("4","Y", "R"),
    
                    ("9","1"):("9","1", "L"),
                        ("9"," "):("0"," ", "R"),
    
 
                    ("11"," "):("13"," ", "R"),
                    ("11","Y"):("13"," ", "R"),
                    ("11","1"):("13"," ", "R"),
                        
    
                    ("12"," "):("12","1", "L"),
                        ("12"," "):("11"," ", "R"),
    
                    ("13"," "):("13"," ", "R"),
                    ("13","1"):("13"," ", "R"),
                    ("13","="):("f"," ", "R"),
                       


                       }
final_states = {"f"}

t = TuringMachine(
                  initial_state = "0",
                  final_states = final_states,
                  transition_function=transition_function, 
                  max_steps = 1000000)

In [38]:

#from_unary(t.debug(f"{to_unary(8)} {to_unary(5)}="))

from_unary(t.process(f"{to_unary(8)} {to_unary(5)}="))

40

In [11]:
transition_function = {
                       ("0","1"):("0", "1", "R"),
                       ("0"," "):("0"," ", "R"),
                        ("0","="):("6","=", "R"),
    
                        ("6","1"):("6","1", "R"),
                        ("6"," "):("7","1", "L"), #Acumule
    
                        ("7","1"):("7","1", "L"),
                        ("7","="):("1","=", "L"),
                       
                       ("1","1"):("1", "Y", "L"),
                       ("1"," "):("2"," ", "R"),
                       
                       ("2","Y"):("3", "1", "L"),
                        ("2"," "):("2", " ", "R"),
                        ("2","1"):("2", "1", "R"),
                       ("2","="):("6","=", "R"),
    
                        ("3","1"):("3","1", "L"),
                        ("3"," "):("4"," ", "L"),
    
                        ("4","1"):("4","1", "L"),
                        ("4"," "):("5"," ", "R"),
                
                        ("5","1"):("2"," ", "R"),
                        ("5"," "):("b"," ", "R"),
    
                        ("b","1"):("b"," ", "R"),
                        ("b"," "):("b"," ", "R"),
                        ("b","Y"):("b"," ", "R"),
                        ("b","="):("b1"," ", "R"),
    
                        ("b1","1"):("f"," ", "R"),
                        ("b1"," "):("f"," ", "R"),
    
                    
    


                       }
final_states = {"f"}

t = TuringMachine(
                  initial_state = "0",
                  final_states = final_states,
                  transition_function=transition_function, 
                  max_steps = 100000)

In [27]:
#from_unary(t.debug(f"{to_unary(13)} {to_unary(5)}="))


In [13]:
# starts accumulating after the =
# changes every 1 from the divisor to y's
# starts changing the y's to 1's, for each such operation deletes a 1 from the dividend
# goes back to delete all the y's
# after all y's are deleted puts 1 to the accumulator and repeats


transition_function = {
                       ("0","1"):("0", "1", "R"),
                       ("0"," "):("0"," ", "R"),
                        ("0","="):("6","=", "R"),
    
                        ("6","1"):("6","1", "R"),
                        ("6"," "):("7","1", "L"), #Acumule
    
                        ("7","1"):("7","1", "L"),
                        ("7","="):("1","=", "L"),
                       
                       ("1","1"):("1", "Y", "L"),
                       ("1"," "):("2"," ", "R"),
                       
                       ("2","Y"):("3", "1", "L"),
                        ("2"," "):("2", " ", "R"),
                        ("2","1"):("2", "1", "R"),
                       ("2","="):("6","=", "R"),
    
                        ("3","1"):("3","1", "L"),
                        ("3"," "):("4"," ", "L"),
    
                        ("4","1"):("4","1", "L"),
                        ("4"," "):("5"," ", "R"),
                
                        ("5","1"):("2"," ", "R"),
                        ("5"," "):("b"," ", "R"),
    
                        ("b","1"):("b1"," ", "R"),
                        ("b"," "):("f"," ", "R"),
    
                        ("b1","1"):("b1","1", "R"),
                         ("b1","Y"):("b1"," ", "R"),
                        ("b1"," "):("b1"," ", "R"),
                        ("b1","="):("b2","=", "R"),
                        ("b2","1"):("f"," ", "N"),
    
                        

                    
    


                       }
final_states = {"f"}

t = TuringMachine(
                  initial_state = "0",
                  final_states = final_states,
                  transition_function=transition_function, 
                  max_steps = 100000)

In [35]:
d = 21
n = 6
#from_unary_div(t.debug(f"{to_unary(d)} {to_unary(n)}="), n)
from_unary_div(t.process(f"{to_unary(d)} {to_unary(n)}="), n)

{'div': 3.5, 'int_div': 3, 'module': 3}

In [34]:
d = 5
n = 10
#from_unary_div(t.debug(f"{to_unary(d)} {to_unary(n)}="), n)
from_unary_div(t.process(f"{to_unary(d)} {to_unary(n)}="), n)

{'div': 0.5, 'int_div': 0, 'module': 5}

In [32]:

#from_unary_div(t.debug(f"{to_unary(5)} {to_unary(0)}="), 10)
from_unary_div(t.process(f"{to_unary(5)} {to_unary(0)}="), 10)

{'div': 222.5, 'int_div': 222, 'module': 5}

In [21]:

#https://stackoverflow.com/questions/59045832/turing-machine-for-addition-and-comparison-of-binary-numbers
#https://www.tutorialspoint.com/design-turing-machine-for-multiplication#